In [1]:
from time import sleep
from random import random
import os, sys
from socket import gethostname
from dask.distributed import Queue
import htpar
reload(htpar)
from random import randint
from distributed import Client, fire_and_forget
# client = Client("192.168.4.3:8786")
client = Client()
client

Client Scheduler: tcp://127.0.0.1:37989,Cluster Workers: 4 Cores: 4 Memory: 33.68 GB


>>> starting <distributed.queues.Queue object at 0x7fce044ce450>
>>> starting <distributed.queues.Queue object at 0x7fce045b2590>
>>> starting <distributed.queues.Queue object at 0x7fce2ef85610>
>>> {'a': '2', 'b': '2', '__key__': '000002'}
>>> {'a': '0', 'b': '0', '__key__': '000000'}
>>> {'a': '5', 'b': '5', '__key__': '000005'}
>>> {'a': '1', 'b': '1', '__key__': '000001'}
>>> {'a': '3', 'b': '3', '__key__': '000003'}
>>> {'a': '6', 'b': '6', '__key__': '000006'}
>>> {'a': '8', 'b': '8', '__key__': '000008'}
>>> {'a': '11', 'b': '11', '__key__': '000011'}
>>> {'a': '4', 'b': '4', '__key__': '000004'}
>>> {'a': '7', 'b': '7', '__key__': '000007'}
>>> {'a': '14', 'b': '14', '__key__': '000014'}
>>> {'a': '17', 'b': '17', '__key__': '000017'}
>>> {'a': '9', 'b': '9', '__key__': '000009'}
>>> {'a': '20', 'b': '20', '__key__': '000020'}
>>> {'a': '10', 'b': '10', '__key__': '000010'}
>>> {'a': '12', 'b': '12', '__key__': '000012'}
>>> {'a': '23', 'b': '23', '__key__': '000023'}
>>> {'a':

In [2]:
def read_fun(queues, src):
    count = 0
    for record in htpar.tarrecords(src):
        shard = count % len(queues)
        queues[shard].put(record)
        count += 1

In [3]:
def write_fun(dest, queue):
    print ">>> starting", queue
    with htpar.TarRecords(dest) as stream:
        while True:
            record = queue.get()
            if record is None: break
            print ">>>", record
            stream.write(record["__key__"], record)
    return "OK"

In [4]:
!echo test | curl -T - http://192.168.4.3/data/temp/_test.txt

curl: (7) Failed to connect to 192.168.4.3 port 80: Connection refused


In [5]:
hs = htpar.HttpStorage()
with hs.open_write("http://192.168.4.6/data/temp/_test.txt") as stream:
    pass

In [6]:
hs = htpar.HttpStorage()
try:
    with hs.open_write("http://192.168.4.3/data/temp/_test.txt") as stream:
        pass
except Exception, e:
    print e

http://192.168.4.3/data/temp/_test.txt: cannot write location


In [7]:
shards = list(htpar.path_shards("http://192.168.4.6/data/temp/_test-@00003.tgz"))
queues = [None] * len(shards)
results = [None] * len(shards)
for i, s in enumerate(shards):
    queues[i] = Queue()
    results[i] = client.submit(write_fun, s, queues[i])
    print i, s, queues[i], results[i]

0 http://192.168.4.6/data/temp/_test-00000.tgz <distributed.queues.Queue object at 0x7fce0406ed10> <Future: status: pending, key: write_fun-8a23641ebc6ab207fc971901c4759730>
1 http://192.168.4.6/data/temp/_test-00001.tgz <distributed.queues.Queue object at 0x7fce2e65a410> <Future: status: pending, key: write_fun-5fc318511281b483b834a13b2a994778>
2 http://192.168.4.6/data/temp/_test-00002.tgz <distributed.queues.Queue object at 0x7fce2e65a250> <Future: status: pending, key: write_fun-f07b2f3b5a7032a750b09c5b9a5d5448>


In [8]:
for i in range(100): 
    queues[i%3].put(dict(a=str(i), b=str(i), __key__="%06d"%i))
for i in range(3): 
    queues[i].put(None)

In [9]:
client.gather(results)

['OK', 'OK', 'OK']